In [1]:
import requests
import time
import re
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import time
import jsondiff
import argparse
import pandas as pd
import os
import subprocess
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
#from openai import OpenAI
from secret import OPENAI_API_KEY
import json, jsondiff
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# ODL Controller Details
ODL_BASE_URL = "http://10.23.7.63:8181/restconf"
USERNAME = "admin"
PASSWORD = "admin"  # Replace with your ODL credentials

# Headers for REST API
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}


# Define sudo password
sudo_password = "your_password"

In [3]:
def get_flows_from_all_nodes_config():
    url = f"{ODL_BASE_URL}/config/opendaylight-inventory:nodes/"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, auth=(USERNAME, PASSWORD), headers=headers)
        response.raise_for_status()
        
        data = response.json()
        node_flow_map = {}

        for node in data.get("nodes", {}).get("node", []):
            node_id = node.get("id", "unknown")
            node_flows = []

            for table in node.get("flow-node-inventory:table", []):
                for flow in table.get("flow", []):
                    node_flows.append(flow.get("id"))
            
            node_flow_map[node_id] = node_flows

        #for node_id, flow_ids in node_flow_map.items():
            #print(f"Node: {node_id}, Flows: {flow_ids}")

        return node_flow_map

    except requests.exceptions.RequestException as e:
        print("Error fetching flows from all nodes in configuration datastore:", e)
        return {}


def get_next_available_flow_id_for_switch(flow_map, node_id):
    """
    Get the next available flow ID for a specific switch (node_id).
    
    Parameters:
        flow_map (dict): Mapping of node IDs to their flow IDs.
        node_id (str): The ID of the target switch.
        
    Returns:
        int: The next available flow ID for the given switch.
    """
    used_ids = set()
    flows = flow_map.get(node_id, [])
    
    for flow in flows:
        try:
            used_ids.add(int(flow))
        except ValueError:
            pass  # Ignore non-integer flow IDs

    return max(used_ids) + 1 if used_ids else 1


def get_flow_details(node_id, flow_id):
    """
    Retrieve details of a specific flow rule by node ID and flow ID from the configuration datastore.
    
    Parameters:
        node_id (str): The ID of the OpenFlow node (e.g., "openflow:1").
        flow_id (str): The ID of the flow rule.
    
    Returns:
        dict: JSON details of the specific flow rule.
    """
    url = f"{ODL_BASE_URL}/config/opendaylight-inventory:nodes/node/{node_id}/table/0/flow/{flow_id}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, auth=(USERNAME, PASSWORD), headers=headers)
        response.raise_for_status()
        
        flow_data = response.json()
        return flow_data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching flow {flow_id} from node {node_id}: {e}")
        return {}


def retrieve_all_flow_details():
    """
    Retrieve all flow rule details from all OpenFlow nodes and save them in a structured format.
    
    Returns:
        dict: Mapping of node IDs to a dictionary of flow IDs and their JSON details.
    """
    flow_details_map = {}
    node_flow_map = get_flows_from_all_nodes_config()

    for node_id, flow_ids in node_flow_map.items():
        flow_details_map[node_id] = {}
        for flow_id in flow_ids:
            flow_details = get_flow_details(node_id, flow_id)
            flow_details_map[node_id][flow_id] = flow_details

    return flow_details_map


def extract_switch_id(intent: str):
    """
    Extract the switch ID from a natural language intent.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'openflow:1') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'openflow:1'
    match = re.search(r'openflow[:\s](\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        return f"openflow:{match.group(1)}"

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return f"openflow:{ordinals[ordinal_word]}"

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"openflow:{number}"

    return None

def extract_port_number(text: str):
    """
    Extract the Ethernet port number from a natural language text.
    
    Parameters:
        text (str): The input text containing the port reference.
    
    Returns:
        int: Extracted port number or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match explicit numbers after keywords
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Match ordinal words (e.g., 'second port', 'third interface')
    match = re.search(r'\b(?:port|interface|output\s+node\s+connector|ethernet)\s*(\w+)', text, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            return ordinals[ordinal_word]

    # Match standalone ordinal words (e.g., 'second')
    for word, number in ordinals.items():
        if word in text.lower():
            return number

    return None


def push_flow_rule(node, table_id, flow_id, flow_json):
    """
    Push a flow rule to ODL config datastore.
    """
    url = f"{ODL_BASE_URL}/config/opendaylight-inventory:nodes/node/{node}/table/{table_id}/flow/{flow_id}"
    
    try:
        response = requests.put(url, headers=HEADERS, auth=(USERNAME, PASSWORD), data=json.dumps(flow_json))
        if response.status_code in [200, 201, 204]:
            print(f"Successfully pushed flow rule to Config Datastore (Switch ID: {node}, Flow ID: {flow_id})")
            return True
        else:
            print(f"Failed to push flow rule to Config Datastore. Status Code: {response.status_code}, Response: {response.text}")
            return False
    except Exception as e:
        print(f"Exception occurred while pushing flow rule: {e}")
        return False


def verify_flow_rule(node, table_id, flow_id):
    """
    Verify if the flow rule exists in ODL operational datastore.
    """
    url = f"{ODL_BASE_URL}/operational/opendaylight-inventory:nodes/node/{node}/table/{table_id}/flow/{flow_id}"
    
    try:
        response = requests.get(url, headers=HEADERS, auth=(USERNAME, PASSWORD))
        if response.status_code == 200:
            print(f"Flow rule exists in Operational Datastore (Flow ID: {flow_id})")
            return True
        else:
            print(f"Flow rule NOT found in Operational Datastore. Status Code: {response.status_code}, Response: {response.text}")
            return False
    except Exception as e:
        print(f"Exception occurred while verifying flow rule: {e}")
        return False

# Function to update flow_id and table_id
def update_flow_fields(flow_json, new_flow_id, new_table_id):
    """
    Update flow_id and table_id in OpenDaylight flow JSON.

    Parameters:
        flow_json (dict): JSON object representing the flow.
        new_flow_id (str): New flow ID to replace the current one.
        new_table_id (int): New table ID to replace the current one.

    Returns:
        dict: Updated JSON object.
    """
    for flow in flow_json.get("flow-node-inventory:flow", []):
        flow["id"] = str(new_flow_id)  # Ensure flow_id is a string
        flow["table_id"] = new_table_id  # Ensure table_id is an integer
    
    return flow_json

def execute_command(command):
    process = subprocess.Popen(command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{sudo_password}\n")
    if process.returncode == 0:
        return stdout.strip()
    else:
        raise Exception(f"Error executing command: {stderr.strip()}")

def get_switch_port_mapping():
    try:
        # Commands to list port and QoS configurations
        list_ports_command = "sudo -S ovs-vsctl list port"
        list_qos_command = "sudo -S ovs-vsctl list qos"
        # Fetch port and QoS data
        ports_output = execute_command(list_ports_command)
        qos_output = execute_command(list_qos_command)

        # Parse QoS data into a dictionary
        qos_mapping = {}
        current_qos = None
        for line in qos_output.splitlines():
            if line.startswith("_uuid"):
                current_qos = line.split(":")[1].strip()
            elif line.startswith("queues") and current_qos:
                qos_mapping[current_qos] = line.split(":")[1].strip()

        # Create a dictionary to store switch-to-port mapping
        switch_port_dict = {}

        # Parse ports data and check for QoS
        current_port = None
        for line in ports_output.splitlines():
            if line.startswith("name"):
                current_port = line.split(":")[1].strip()
            elif line.startswith("qos") and "[]" not in line and current_port:
                qos_uuid = line.split(":")[1].strip()

                # Extract the OpenFlow switch ID and port number
                if "-" in current_port:
                    switch, port = current_port.split("-")
                    switch_id = f"openflow:{switch[1:]}"  # e.g., "s1" -> "openflow:1"
                    port_number = port[3:]  # e.g., "eth2" -> "2"

                    # Add to dictionary
                    if switch_id not in switch_port_dict:
                        switch_port_dict[switch_id] = []
                    switch_port_dict[switch_id].append(port_number)

                current_port = None

        return switch_port_dict

    except Exception as e:
        print(f"Error: {e}")
        return {}


def create_two_queue_for_switch(switch, port, max_rate=10000000, queue_configs=None):
    """
    Creates queues dynamically for a specific switch and port.
    
    Parameters:
        switch (str): The name of the switch in 'openflow:X' format (e.g., 'openflow:4').
        port (int): The port number on the switch (e.g., 2).
        max_rate (int): Maximum rate for the QoS (default is 10000000).
        queue_configs (list): List of tuples specifying min-rate and max-rate for each queue (default is 2 queues).
    """
    if queue_configs is None:
        # Default to 2 queues with these configurations
        queue_configs = [
            (6000000, 6000000),  # Queue 0: min-rate and max-rate
            (4000000, 4000000)   # Queue 1: min-rate and max-rate
        ]

    # Construct the port name from the input
    port_name = f"{switch.replace('openflow:', 's')}-eth{port}"

    # Construct the QoS command for the specific switch and port
    qos_command = f"sudo -S ovs-vsctl -- set port {port_name} qos=@newqos -- --id=@newqos create qos type=linux-htb other-config:max-rate={max_rate}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" queues:{i}=@q{i}"
    for i, (min_rate, max_rate) in enumerate(queue_configs):
        qos_command += f" -- --id=@q{i} create queue other-config:min-rate={min_rate} other-config:max-rate={max_rate}"

    # Execute the command
    print(f"Running: {qos_command}")
    process = subprocess.Popen(qos_command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{sudo_password}\n")
    if process.returncode == 0:
        print(f"Success:\n{stdout}")
    else:
        print(f"Error:\n{stderr}")

In [ ]:
TRANSLATION_PROMPT = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the OpenDaylight (ODL) SDN controller's configuration datastore.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

a. **Traffic Forwarding and Queue-Based Traffic Control Rule:** Define rules for forwarding traffic based on IPv4 destination, TCP/UDP ports, and optionally assign traffic to specific queues for prioritization. 
b. **Firewall and Blocking:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP).  
c. **Port-Based Forwarding:** Redirect traffic entering a specific port to another designated port.

### JSON STRUCTURE:

1. **Traffic Forwarding and Queue-Based Traffic Control Rule:**  

```json
{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": 2048
          }
        },
        "ipv4-source": "<ip_address/mask>", //**optional**
        "ipv4-destination": "<ip_address/mask>", //**optional**
        "ip-match": {
          "ip-protocol": <integer>
        },
        "tcp-destination-port": <integer>,
        "udp-destination-port": <integer>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "set-queue-action": {
                    "queue-id": <queue_id>
                  }
                },
                {
                  "order": 1,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

2. **Blocking or Dropping Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": <integer>
          }
        },
        "ipv4-source": "<ip_address/mask>",
        "ipv4-destination": "<ip_address/mask>"
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "drop-action": {}
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

3. **Port-Based Forwarding Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name_summerizing_the_intent>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "in-port": <port_number>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

Field Descriptions: 
id: A number representing a unique identifier for the flow (0 for default).
priority: Priority level (higher numbers indicate higher priority). For dropping or blocking or firewall rule, assign priority greater than 300.
table_id: An integer representing the flow table identifier (0 for default).
flow-name: A short, descriptive flow name that summerizes the intent.
hard-timeout: Timeout in seconds after which the flow is removed (0 for default).
idle-timeout: Timeout in seconds after which the flow is removed if there's no activity (0 for default).
ethernet-type: Ethernet protocol type (e.g., 2048 for IPv4).
ipv4-destination: IPv4 address in CIDR notation (e.g., 10.0.0.1/32). Note: This field is optional. Don't include it unless IP address (e.g., 10.0.0.1) is explicitly mentioned in the intent.
ipv4-source: Source IP address (optional). Note: This field is optional. Don't include it unless IP address (e.g., 10.0.0.1) is explicitly mentioned in the intent.
ip-protocol: Use "ip-match" and "ip-protocol" when specifying specific transport layer protocols (e.g., 6 for TCP, 17 for UDP, 1 for ICMP (optional)).
tcp-source-port: The source port for the connection, usually a random high port on the client (rarely fixed) (optional).
tcp-destination-port: The destination port for the connection (i.e., the server port, e.g., 80 for HTTP) (optional).
udp-source-port: UDP port number (optional).
udp-destination-port: UDP port number (optional).
in-port: A value representing incoming interface port number (optional).
output-action: Use "output-action" and "output-node-connector" to specify the output port number (optional).
set-queue-action: Use "set-queue-action" and "queue-id" when the intent specifies assigning traffic to a queue (The "queue-id" is an integer (0 for default)).
drop-action: Use {} to indicate packet dropping.

RULES:
Each id must be unique.
Set priority values appropriately (higher for critical rules, lower for defaults). Set priority very high (e.g., 500) for queue related rules.
Don't include any **optional field** unless it is explicitly mentioned in the intent.
Use valid match conditions (ipv4-destination, tcp-destination-port, ipv4-source, in-port) depending on the intent type.
When translating HTTP, HTTPS, or other protocol traffic by port, always use 'tcp-destination-port' for the protocol's standard server port (e.g., 80 for HTTP, 443 for HTTPS) unless the intent explicitly says 'source port'.
Ensure valid ODL-compliant JSON syntax.
Avoid duplicate keys and empty fields.
Verify JSON structure for correctness before responding.
Always respond with valid JSON only, with no additional text, comments, or explanations.
If the intent cannot be mapped, return an empty JSON object {}."""

In [5]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in a openflow switch. You should respond in JSON format. 

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`. 

2. **Switch and Port Identification:**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates output port or outgoing interface.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>"
}

Field Description:
-use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
-switch_id: Switch ID if specified in the intent, otherwise "".
-queue_id: Queue ID if specified in the intent, otherwise "".
-port_id: Port ID if specified in the intent, otherwise "". If there are mutiple instances of "port_id" present, take the one which indicates the output port or outgoing interface.

4. No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:
    1. Input Intent:
    "Create a queue in switch 4 on port 3 for slicing the flow."
    Output:
    {
    "use_queue": 1,
    "switch_id": "switch 4",
    "queue_id": "",
    "port_id": "port 3"
    }

    2. Input Intent:
    "Send all video traffic through queue 0 of openflow:2."
    Output:
    {
    "use_queue": 1,
    "switch_id": "openflow 2",
    "queue_id": 0,
    "port_id": ""
    }

    3. Input Intent:
    "Configure switch 5 for traffic management."
    Output:
    {
    "use_queue": 0,
    "switch_id": "switch 5",
    "queue_id": "",
    "port_id": ""
    }

    4. Input Intent:
    "Monitor traffic flow on port 1."
    Output:
    {
    "use_queue": 0,
    "switch_id": "",
    "queue_id": "",
    "port_id": "port 1"
    }

   5. Input Intent:
    "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
    Output:
    {
    "use_queue": 1,
    "switch_id": switch 3,
    "queue_id": 0,
    "port_id": "interface 2"
    }

Respond with the appropriate JSON strictly following these rules and format."""

In [ ]:
CONFLICT_PROMPT = """You are tasked with determining if two OpenDaylight (ODL) flow configuration JSONs directly conflict with each other. You MUST base your decision SOLELY on the content of the JSON structures provided. Do NOT consider the intent behind the flow rule or try to infer conflicts based on the general idea of blocking or forwarding traffic. Your analysis MUST be a strict, field-by-field comparison of the JSON structures.

### **Rules of Conflict Detection (STRICT and LITERAL):


A **direct conflict** exists ONLY if *all* of the following conditions are met:

1. **Exact Matching Traffic Characteristics (Match Criteria Overlap):** Both flows MUST have *identical* or *overlapping* match criteria for *all specified fields*. This means they could potentially apply to the *exact same* packet. Critically, if a field is present in one flow but *not* in the other, there is *NO MATCH* for that field. Consider the following match fields:
    *   `in-port`: Both flows must have the `in-port` field specified and the values must be identical for an overlap. If either flow does not have the `in-port` field, there is NO overlap for this criterion.
    *   `ethernet-match.ethernet-type.type`: Both flows must have this field and the values must be identical (e.g., 2048 for IPv4) for an overlap. If either flow does not have this field, there is NO overlap for this criterion.
    *   `ipv4-source`: Both flows must have the `ipv4-source` field specified. The source IP addresses/masks must overlap (e.g., 10.0.0.1/32 and 10.0.0.0/24 overlap) for an overlap. If either flow does not have the `ipv4-source` field, there is NO overlap for this criterion.
    *   `ipv4-destination`: Both flows must have the `ipv4-destination` field specified. The destination IP addresses/masks must overlap for an overlap. If either flow does not have the `ipv4-destination` field, there is NO overlap for this criterion.
    *   `ip-match.ip-protocol`: Both flows must have the `ip-protocol` field specified and the values must be identical (e.g., 6 for TCP, 17 for UDP) for an overlap. If either flow does not have this field, there is NO overlap for this criterion.
    *   `tcp-source-port`, `tcp-destination-port`, `udp-source-port`, `udp-destination-port`: Both flows must have the *same* port field (e.g., both `tcp-destination-port`) specified, and the values must be identical for an overlap. If either flow does not have the corresponding port field, there is NO overlap for this criterion.
    *    **EXCEPTION** If one flow has additional match fields for ethernet-type or ip-protocol or tcp-destination-port or udp-destination-port, then there is NO overlap for these flows even if the output actions are same and some but not all required fields (such as ethernet-type, ip-protocol, IP address, tcp-destination-port, udp-destination-port) match.

2. **Contradictory Actions:** If *and only if* the match criteria overlap as described above, check the actions. Contradictory actions include:
    *   Different `output-node-connector` values: If the flows have different output ports, it's a conflict..
    *   One flow has `drop-action: {}` and the other has an `output-action`: If one flow drops the packet and the other forwards it, it's a conflict.
    *   Different `set-queue-action.queue-id` values: If the flows specify different queue IDs, it's a conflict.
    *   **EXCEPTION** If one flow has additional match fields for ethernet-type or ip-protocol or tcp-destination-port or udp-destination-port, then there is NO overlap for these flows even if the output actions are same and some but not all required fields (such as ethernet-type, ip-protocol, IP address, tcp-destination-port, udp-destination-port) match.

3. **Priority is Irrelevant for Direct Conflicts:** The `priority` field does *not* determine if a direct conflict exists. Priority only determines which flow rule takes precedence if a conflict is detected by the switch.

4. **No General vs. Specific Rule Heuristics**  
    Do NOT assume that one rule is “more general” or “more specific.” Only literal match-set overlap matters. Two rules conflict if — and only if — they match the same packet and specify different actions.
    
5. **Source to Source, Destination to Destination:** You MUST match source to source and destination to destination. Any match between a JSON's source to another JSON's destination is NOT a conflict.

6. **Missing Field Handling - EXTREMELY IMPORTANT (LITERAL COMPARISON):**
    *   If one JSON omits a field (e.g., ipv4-source), it matches all values for that field. Therefore, if the other JSON specifies a narrower match, and their actions differ, this can be a conflict.
    *   **Crucially:** You MUST perform a literal, field-by-field comparison. Do NOT infer any broader meaning or intent.
    *   **Example 1 (CONFLICT):**
        *   Flow 1: `{"match": {"ipv4-destination": "10.0.0.2/32"}, "instructions": { ... "output-action": ... }}`
        *   Flow 2: `{"match": {"ipv4-source": "10.0.0.1/32", "ipv4-destination": "10.0.0.2/32"}, "instructions": { ... "drop-action": {} }}`
        *   **Explanation:** Flow 1 matches all sources to 10.0.0.2. Flow 2 matches only packets from 10.0.0.1 to 10.0.0.2. Their match sets may overlap — so if their actions differ, this is a conflict. 
   *     **Example 2 (NO CONFLICT):**
         *  One flow allows traffic to `10.0.0.2/32` 
         *  Another flow blocks traffic from `10.0.0.2/32` to `10.0.0.4/32`
         *   **Explanation:** These do not conflict because source and destination fields do not align.
   *   **Example 3 (NO CONFLICT):**
        *   Flow 1: `{"match": {"ipv4-source": "10.0.0.1/32", "ipv4-destination": "10.0.0.4/32"}, "instructions": { ... "drop-action": {} }}`
        *   Flow 2: `{"match": {"ipv4-source": "10.0.0.2/32", "ipv4-destination": "10.0.0.4/32"}, "instructions": { ... "drop-action": {} }}`
        *   **Explanation:** Although both flows drop traffic to 10.0.0.4, they have *different sources*. Therefore, they do NOT conflict. They apply to different traffic.
   *     **Example 4 (NO CONFLICT):**
         *  One flow has destination IP `10.0.0.1` 
         *  Another flow has detination IP `10.0.0.2`
         *   **Explanation:** These do not conflict even if the output actions are different because the source and the destination IP are not same, i.e. 10.0.0.1/32 and 10.0.0.2/32 do not overlap as 10.0.0.1 is not within the subnet of 10.0.0.2/32.

### **Input Format:**

You will be provided with two JSON objects representing ODL flow entries. The input will be formatted as follows:

Flow 1:
<JSON for Flow 1>

Flow 2:
<JSON for Flow 2>

### **Expected Output Format:**

Respond strictly in valid JSON adhering to the following schema:

```json
{
  "conflict_status": <integer>,
  "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:

conflict_status: An integer value: 1 if a direct conflict exists, 0 if no direct conflict exists. **DO NOT PUT 1 IF YOU ARE NOT CONFIDENT ABOUT A DIRECT CONFLICT**.
conflict_explanation: A brief explanation of the detected conflict, if any. This field should be an empty string ("") if there is no conflict.
Do not include any additional text, comments, or explanations outside of the JSON structure. Only output valid JSON."""


In [ ]:
my_models = [
"qwq"
    ]
  
context_examples = [3]

default_model = "qwq"

ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [ ]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('Intent2Flow-ODL.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

trainset = custom_dataset

In [10]:
result_dict_error_count = {}

In [ ]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json):

    system_prompt = CONFLICT_PROMPT

    print("\nCheckpoint*******Entering RUM LLM Conflict\n\n******")
    
    for model in my_models:     
        
        while True:
            try:
                time.sleep(0.1)             
                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    #options={'device': 'cpu'},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )
                
                output = response['response'].strip()
                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    print("\nWarning: 'conflict_status' key is missing in the response.\n")
                
                conflict_status = response_json.get('conflict_status', 0)
                
                if not isinstance(conflict_status, int):
                    if (type(conflict_status).__name__ == 'str'):
                        conflict_status = int(conflict_status)
                
                if conflict_status == 1:
                    return True, response_json['conflict_explanation']
                
                break
                
                #result_dict_error_count[model] = result_dict_error_count.get(model, 0) + conflict_status
                #print("\n\nConflict Status by: ", model, "\n\n", output)               
                #break
            
            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                continue      
    print("\nCheckpoint*******Exiting RUN LLM Conflict\n\n******")
    #print("\n\n============Trying another flow==============\n\n")
    return False, None

In [12]:
def conflict(node_id, new_intent_flow_json):

    all_flow_details = retrieve_all_flow_details()
    flows_of_node_id = all_flow_details.get(node_id, {})

    print("\nCheckpoint*******Inside Conflict Checking\n\n******")

    for flow_id, existing_intent_flow_json in flows_of_node_id.items():
        print("\n",node_id, " ", flow_id,"\n")
        conflict_status, conflict_info = run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json)
        if conflict_status == True :
            return True, flow_id, conflict_info
    
    #for model, errors in result_dict_error_count.items():
        #print(f"{model}: {errors}")
    
    #return True, None, None
    print("\nCheckpoint*******Exiting conflict Checking\n\n******")
    return False, None, None

In [13]:
def run_LLM_Slice(intent):

    system_prompt = SLICING_PROMPT
    
    for model in my_models:     
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system=system_prompt,
                prompt=intent,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            
            print("\nCheckpoint*******Exiting LLM Slicing detection\n\n******")
            return response_json            
        
        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            continue

In [14]:
def run_LLM_IBN(intent, node_id):

    for num_examples in context_examples:
        for model in my_models:
            example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )

            # Clear and add all remaining examples from the trainset
            example_selector.vectorstore.reset_collection()
            for _, row in trainset.iterrows():
                example_selector.add_example({
                    "instruction": row["instruction"],
                    "output": row["output"]
                })
            
            system_prompt = TRANSLATION_PROMPT

            while True:
                try:
                    time.sleep(0.1)
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        #options={'device': 'cpu'},
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    print("\nTranslated by: ", model)
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    sys.stdout.flush()
                    continue
            
            try:
                
                print("\nCheckpoint*******Entering next flow id checking\n\n******")
                new_flow_id = get_next_available_flow_id_for_switch(get_flows_from_all_nodes_config(), node_id)
                new_table_id = 0
                
                flow_json = json.loads(actual_output)
                updated_flow_json = update_flow_fields(flow_json, new_flow_id, new_table_id)

                print("\n\n", intent, "\n", flow_json, "\n\n")

                print("\nCheckpoint*******Entering conflict detection\n\n******")
                conflict_status, flow_id, conflict_info = conflict(node_id, updated_flow_json)

                print("\nCheckpoint*******Conflict detection done\n\n******")

                if(conflict_status == True):
                    print("\n\nThe new intent JSON conflicts with an existing intent JSON\nCheck switch: ",node_id, ", flow id: ",flow_id, "\n\n", conflict_info)
                    return False
                else:
                    # Push Flow Rule
                    print("\nCheckpoint*******Entering PUSH flow rule of ODL\n\n******")
                    if push_flow_rule(node_id, new_table_id, new_flow_id, updated_flow_json):
                        print("Waiting for the rule to propagate to Operational Datastore...\n")
                        time.sleep(5)  # Introduce a 5-second delay
                        # Verify Flow Rule
                        if verify_flow_rule(node_id, new_table_id, new_flow_id):
                            print("\n\n", intent, "\n", flow_json, "\n\n")
                            return True
            
            except Exception as e:
                print("Exception found: ", e)
    print("\nCheckpoint*******Exiting IBN LLM\n\n******")
    return False 

In [15]:
def main():

    intent = input().strip()
    #intent = "in switch 4, install a firewall to block traffic from 10.0.0.2 to 10.0.0.4"
    #intent = "in switch 4, traffic destined for 10.0.0.4 should pass through port 3"
    #intent = "Using openflow switch number 1, forward UDP traffic on port 80 destined for 10.0.0.3 via interface 2, assigning it to queue 1 for prioritized handling."
    #intent = "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
    #intent = "In switch 3, if the incoming traffic in interface 1 is UDP traffic destined for port 80, then pass it via port 2, assigning it to queue 1 for prioritized handling."
    #intent = "In switch 3, if the incoming traffic in port 2 is TCP traffic destined for port 80, then pass it via interface 1, assigning it to queue 0 for prioritized handling."
    #intent = "In switch 3, if the incoming traffic in interface 2 is UDP traffic destined for port 80, then pass it via port 1, assigning it to queue 1 for prioritized handling."

    current_time = time.time()

    openflow_id = extract_switch_id(intent)

    print("\nCheckpoint*******Entering LLM Slicing detection\n\n******")
    sliceing_info = run_LLM_Slice(intent)

    if 'use_queue' in sliceing_info:         
        slicing_status = sliceing_info['use_queue']
        slicing_switch_id = sliceing_info['switch_id']
        slicing_queue_id = sliceing_info['queue_id']
        slicing_port_id = sliceing_info['port_id']

        if(slicing_status == 1):
            openflow_id = extract_switch_id(slicing_switch_id)
            switch_port_mapping = get_switch_port_mapping()
            print("\nCheckpoint*******Entering Slice/Queue Management\n\n******")

            port_number = extract_port_number(slicing_port_id)
            
            if openflow_id not in switch_port_mapping :
                
                    print("\n\nQueue was not installed in ",openflow_id, "\nInstalling now on interface: ", port_number,"\n")
                    print("\nCheckpoint*******Entering queue creation\n\n******")

                    create_two_queue_for_switch(
                        switch=openflow_id,  port=port_number,
                        queue_configs=[
                            (6000000, 6000000),  # Queue 0
                            (4000000, 4000000)   # Queue 1
                        ]
                        )   
            else:
                if str(port_number) not in switch_port_mapping[openflow_id]:

                    print("\n\nQueue was not installed in ",openflow_id, " interface: ", port_number, "\nInstalling now...\n")
                    print("\nCheckpoint*******Entering queue creation\n\n******")

                    create_two_queue_for_switch(
                        switch=openflow_id,  port=port_number,
                        queue_configs=[
                            (6000000, 6000000),  # Queue 0
                            (4000000, 4000000)   # Queue 1
                        ]
                        )

    if run_LLM_IBN(intent, openflow_id):
        proc_time_s = (time.time() - current_time)
        print("\n\nSuccessfully translated and installed the rule in ODL SDN Controller. Time taken: ", proc_time_s)
    else:
        print("\n\nFailed to install the intent.\n")

In [ ]:
if __name__ == "__main__":
    main()